<a href="https://colab.research.google.com/github/vasan12sp/DocSummarizer/blob/master/Improved_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install transformers datasets rouge-score torch evaluate


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=aea8b371538a63f8523b2694856c90d2b3e2bf0f565ba720ad8929cd63dd8fdf
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dep

In [ ]:

# Import libraries
import torch
import random
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    XLNetTokenizer, XLNetForSequenceClassification,
    BartTokenizer, BartForConditionalGeneration,
    AdamW
)
from torch.cuda.amp import GradScaler, autocast
from evaluate import load



In [ ]:
# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Randomly sample 10,000 articles from the train set and 500 from the validation set
train_sample = random.sample(range(len(dataset['train'])), 10000)
val_sample = random.sample(range(len(dataset['validation'])), 500)

train_dataset = dataset['train'].select(train_sample)
val_dataset = dataset['validation'].select(val_sample)

print(f"Training on {len(train_dataset)} articles")
print(f"Evaluating on {len(val_dataset)} articles")

# Preprocessing function
def preprocess_function(examples):
    inputs = examples['article']
    labels = examples['highlights']
    return {"input_text": inputs, "target_text": labels}

# Tokenize datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# Create DataLoaders
train_dataloader = DataLoader(tokenized_train, batch_size=4, shuffle=True)
val_dataloader = DataLoader(tokenized_val, batch_size=4, shuffle=False)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Training on 10000 articles
Evaluating on 500 articles


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
class Summ_xlnet_bart:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Load XLNet
        self.xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
        self.xlnet_model = XLNetForSequenceClassification.from_pretrained(
            'xlnet-base-cased', num_labels=1
        ).to(self.device)

        # Load BART
        self.bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
        self.bart_model = BartForConditionalGeneration.from_pretrained(
            'facebook/bart-large-cnn'
        ).to(self.device)

        # Load the saved model weights
        model_state = torch.load('/content/drive/My Drive/summ_xlnet_bart.pth')
        self.xlnet_model.load_state_dict(model_state['xlnet_state'])
        self.bart_model.load_state_dict(model_state['bart_state'])

        # Set models to training mode
        self.xlnet_model.train()
        self.bart_model.train()

        # Mixed precision training
        self.scaler = GradScaler()

    def train(self, train_dataloader, val_dataloader, epochs=5, learning_rate=5e-5):
        optimizer = AdamW(
            list(self.xlnet_model.parameters()) + list(self.bart_model.parameters()),
            lr=learning_rate
        )

        loss_fn = torch.nn.BCEWithLogitsLoss()

        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")

            total_loss = 0
            self.xlnet_model.train()
            self.bart_model.train()

            progress_bar = tqdm(train_dataloader, desc="Training", leave=False)

            for batch_idx, batch in enumerate(progress_bar):
                inputs = batch['input_text']
                labels = batch['target_text']

                # Tokenize inputs for XLNet
                xlnet_inputs = self.xlnet_tokenizer(
                    inputs, padding=True, truncation=True,
                    return_tensors='pt', max_length=512
                ).to(self.device)

                # Mixed precision training with autocast
                with autocast():
                    outputs = self.xlnet_model(**xlnet_inputs)
                    importance_scores = outputs.logits.squeeze()

                    # Compute loss
                    loss = loss_fn(importance_scores, torch.ones_like(importance_scores))

                # Backpropagation with mixed precision
                optimizer.zero_grad()
                self.scaler.scale(loss).backward()
                self.scaler.step(optimizer)
                self.scaler.update()

                total_loss += loss.item()
                progress = f"Loss: {loss.item():.4f}"
                progress_bar.set_description(progress)

            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Avg Training Loss: {avg_train_loss:.4f}")

            # Validation after every epoch
            self.evaluate(val_dataloader)

    def evaluate(self, val_dataloader):
        rouge = load("rouge")
        predictions = []
        references = []

        with torch.no_grad():
            self.xlnet_model.eval()
            self.bart_model.eval()

            for batch in val_dataloader:
                inputs = batch['input_text']
                labels = batch['target_text']

                xlnet_inputs = self.xlnet_tokenizer(
                    inputs, padding=True, truncation=True,
                    return_tensors='pt', max_length=512
                ).to(self.device)

                outputs = self.xlnet_model(**xlnet_inputs)
                importance_scores = outputs.logits.squeeze().cpu().numpy()
                top_sentences = np.argsort(importance_scores)[-3:]

                extractive_summary = ". ".join([inputs[0].split('.')[i] for i in top_sentences]) + '.'

                bart_input = self.bart_tokenizer(
                    extractive_summary, return_tensors='pt',
                    max_length=1024, truncation=True
                ).to(self.device)

                generated_summary_ids = self.bart_model.generate(
                    **bart_input, num_beams=4, max_length=150, early_stopping=True
                )

                generated_summary = self.bart_tokenizer.decode(
                    generated_summary_ids[0], skip_special_tokens=True
                )

                predictions.append(generated_summary)
                references.append(labels[0])

        rouge_scores = rouge.compute(predictions=predictions, references=references)
        print(f"ROUGE Scores: {rouge_scores}")



In [ ]:
# Initialize the model and continue training
model = Summ_xlnet_bart()
model.train(train_dataloader, val_dataloader, epochs=5)

# Save the updated model
model_state = {
    'xlnet_state': model.xlnet_model.state_dict(),
    'bart_state': model.bart_model.state_dict()
}
torch.save(model_state, '/content/drive/My Drive/summ_xlnet_bart_updated.pth')
print("Model saved to /content/drive/My Drive/summ_xlnet_bart_updated.pth")


Using device: cuda


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

...

ROUGE Scores: {'rouge1': 0.3823671807951099, 'rouge2': 0.16133440239146038, 'rougeL': 0.26253707902486006, 'rougeLsum': 0.326244085643826}
Epoch 2/5


Avg Training Loss: 0.0000
ROUGE Scores: {'rouge1': 0.3796309279498907, 'rouge2': 0.15930613406471392, 'rougeL': 0.26053716718177605, 'rougeLsum': 0.32336800866554427}
Epoch 3/5


Avg Training Loss: 0.0000
ROUGE Scores: {'rouge1': 0.38272360106144543, 'rouge2': 0.16095205829499987, 'rougeL': 0.2630325968215641, 'rougeLsum': 0.32645790773379013}
Epoch 4/5


Avg Training Loss: 0.0000
ROUGE Scores: {'rouge1': 0.3825468573628026, 'rouge2': 0.16063572512674443, 'rougeL': 0.26253400885751044, 'rougeLsum': 0.32613271616451156}
Epoch 5/5


Avg Training Loss: 0.0000
